<a href="https://colab.research.google.com/github/FinnRobertson15/PatternAnalysis-2023/blob/topic-recognition/Colab%20version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/AD_NC

/content/drive/MyDrive/AD_NC


In [3]:
%ls

NC.zip  test/  train/


In [ ]:
!unzip NC.zip

unzip:  cannot find or open NC.zip, NC.zip.zip or NC.zip.ZIP.


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import time
import numpy as np
# import pandas as pd
import math
import os
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset, ConcatDataset

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_paths[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image

class SiameseDataset(Dataset):
    def __init__(self, AD, NC):
        self.AD = AD
        self.NC = NC
        self.labels = torch.cat((torch.ones(len(AD)), torch.zeros(len(NC))), dim=0)

    def __len__(self):
        return min(len(self.AD), len(self.NC))

    def __getitem__(self, idx):
        img1 = self.AD[idx]
        label1 = self.labels[idx]
        img2 = self.NC[idx]
        label2 = self.labels[len(self.AD) + idx]

        return img1, img2, label1, label2

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


size = 256
transform_X = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.ToTensor(),
    # transforms.Lambda(lambda x: x / 255.0),
    # transforms.Normalize(mean=[0.5], std=[0.5])  # Adjust according to your data statistics
])


# Replace 'your_nii_folder' with the path to your folder containing .nii files
# nii_dataset = NiiDataset(root_dir=r'keras_png_slices_data\keras_png_slices_data\keras_png_slices_train')
loaders = {}

for stage in ['test']:
    loaders[stage] = {}
    AD = CustomDataset(root_dir=os.path.join(stage, 'AD'), transform=transform_X)
    NC = CustomDataset(root_dir=os.path.join(stage, 'NC'), transform=transform_X)
    loaders[stage] = SiameseDataset(AD, NC)

In [6]:
for load in loaders.values():
  print(len(load))

0
4460


In [8]:
import torch
import torch.nn as nn
import torchvision.models as models

class SiameseNetwork(nn.Module):
    def __init__(self, pretrained=True):
        super(SiameseNetwork, self).__init__()
        self.resnet = models.resnet18(pretrained=pretrained)
        # Modify the first convolution layer to accept single-channel input
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.linear = nn.Linear(1000, 2)  # 1000 is the number of output features from ResNet

    def forward(self, x1, x2):
        output1 = self.resnet(x1)
        output2 = self.resnet(x2)
        return self.linear(torch.abs(output1 - output2))

model = SiameseNetwork()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                                      label * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

In [12]:
# Initialize the network, loss function, and optimizer
siamese_net = SiameseNetwork(pretrained=True).to(device)
criterion = ContrastiveLoss()
optimizer = optim.Adam(siamese_net.parameters(), lr=0.0005)

# Training loop
epochs = 2
total_step = len(loaders['test'])
for epoch in range(epochs):
  for i, (i1, i2, l1, l2) in enumerate(loaders['test']):
    i1, i2, l1, l2 = i1.to(device), i2.to(device), l1.to(device), l2.to(device)
    i1 = i1.unsqueeze(1)  # Add a channel dimension
    i2 = i2.unsqueeze(1)  # Add a channel dimension
    output = siamese_net(i1, i2)
    o1, o2 = output[0][0], output[0][1]
    l = (l1 == l2).int()


    # Adjust the shapes to match the criterion
    # outputs = outputs.view(-1)  # Flatten the outputs to a 1D tensor
    # seg = seg.view(-1)  # Flatten the seg tensor to a 1D tensor
    loss = criterion(o1, o2, l)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 500 == 0:
      print(f"Epoch [{epoch + 1} / {epochs}], Step [{i + 1} / {total_step} Loss {loss.item()}]")

Epoch [1 / 2], Step [500 / 4460 Loss 3.259337688632513e-07]
Epoch [1 / 2], Step [1000 / 4460 Loss 3.283045589341782e-05]
Epoch [1 / 2], Step [1500 / 4460 Loss 1.054049062076956e-05]
Epoch [1 / 2], Step [2000 / 4460 Loss 1.234111095982371e-06]
Epoch [1 / 2], Step [2500 / 4460 Loss 1.1719105259544449e-06]
Epoch [1 / 2], Step [3000 / 4460 Loss 3.3835010526672704e-07]
Epoch [1 / 2], Step [3500 / 4460 Loss 7.921468636595819e-08]
Epoch [1 / 2], Step [4000 / 4460 Loss 2.9854402328055585e-07]
Epoch [2 / 2], Step [500 / 4460 Loss 1.767582347156349e-07]
Epoch [2 / 2], Step [1000 / 4460 Loss 4.3589712395153413e-10]
Epoch [2 / 2], Step [1500 / 4460 Loss 8.744237902647001e-07]
Epoch [2 / 2], Step [2000 / 4460 Loss 2.0802056258095725e-11]
Epoch [2 / 2], Step [2500 / 4460 Loss 1.8013805913597025e-07]
Epoch [2 / 2], Step [3000 / 4460 Loss 3.1853244308877038e-06]
Epoch [2 / 2], Step [3500 / 4460 Loss 2.546377402268263e-07]
Epoch [2 / 2], Step [4000 / 4460 Loss 2.075099914122802e-08]


In [18]:
# Evaluation loop
siamese_net.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation for evaluation
    correct = 0
    total = 0
    for i, (i1, i2, l1, l2) in enumerate(loaders['test']):
        i1, i2, l1, l2 = i1.to(device), i2.to(device), l1.to(device), l2.to(device)
        i1 = i1.unsqueeze(1)  # Add a channel dimension
        i2 = i2.unsqueeze(1)  # Add a channel dimension
        output = siamese_net(i1, i2)
        o1, o2 = output[0][0], output[0][1]
        l = (l1 == l2).int()
        # Compute the Euclidean distance between the outputs
        distance = torch.abs(o1 - o2)

        # Apply threshold to determine predicted labels
        threshold = 0.5  # Adjust as needed
        predicted = distance.lt(threshold)



        total += l
        correct += (predicted == l).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')


Test Accuracy: nan%
